## Подключим всё необходимое

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import json
import pymorphy2
from nltk.stem.snowball import SnowballStemmer 
from nltk.corpus import stopwords 
import nltk

from sklearn.model_selection import GroupKFold, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score, make_scorer

In [2]:
import html2text
from nltk.stem.snowball import SnowballStemmer 

In [102]:
#lemmatizer = pymorphy2.MorphAnalyzer(lang = 'ru')
#lemmatizer_cache = {}


#def lemmatize(token):
#    if len(token) < 3:
#        return token
#    if lemmatizer.word_is_known(token):
#        if token not in lemmatizer_cache:
#            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
#        return lemmatizer_cache[token]
#    return token


def lemmatize(token):
    if len(token) < 3:
        return token
    stemmer = SnowballStemmer("russian")
    token = stemmer.stem(token)
    return token

def lemmatize1(token):
    if len(token) < 3:
        return token
    stemmer = SnowballStemmer("english")
    token = stemmer.stem(token)
    return token

In [105]:
import re
set_of_stopwords = set(stopwords.words('russian') + stopwords.words('english'))

def nlp(input_data):
    if type(input_data) != str:
        text = " ".join(input_data)
    else:
        text = input_data
    text = text.lower()
    #tokens = nltk.word_tokenize(text, language = 'russian')
    words_ru = re.findall(r'\b[а-я,ё]+\b', text)
    words_eng = re.findall(r'\b[a-z]+\b', text)
    words_ru = [lemmatize(word) for word in words_ru]
    words_eng = [lemmatize1(word) for word in words_eng]
    words = words_ru + words_eng
    res = []
    for word in words:
        if word not in set_of_stopwords:
            res.append(word)
    return res

In [106]:
nlp(["hello", "worlds"])

['hello', 'world']

In [146]:
import os
directory='content'
files = os.listdir(directory)


In [147]:
files[0]

'1.dat'

In [148]:
i = 1

In [149]:
%%time
texts = []
urls = []
for file in files:
    if i % 100 == 0:
        print(i)
    f = open('content/' + file, 'r', encoding='utf-8')
    url = nlp(f.readline().strip())
    urls.append(url)
    #x = f.read()
    #try:
    #    y = nlp(html2text.html2text(x))
    #except:
    #    y = nlp(x)
    #    print("bad page", i)
    #texts.append(y)
    i = i + 1

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [151]:
len(urls)

28026

In [152]:
urls[27037]

['edimka', 'ru', 'cgi', 'bin', 'cm', 'pl', 'ps', 'r']

In [153]:
#with open('pages.json', 'w', encoding='utf-8') as file:
#    json.dump(texts, file)

In [154]:
#%%time
#with open('pages.json', 'r', encoding='utf-8') as file:
#    json_data = json.load(file)

In [155]:
docs_titles_path = 'docs_titles.tsv'
docs_parsing_json = 'out.json'
train_data = pd.read_csv('train_groups.csv')
test_data = pd.read_csv('test_groups.csv')
docs_titles_data = pd.read_csv(docs_titles_path, sep='\t')

### Достанем из json файла данные

In [156]:

#with open(docs_parsing_json) as f:
#    json_data = json.load(f)

In [157]:
nlp("приветы пока"), nlp(["приветы" ,"пока"])

(['привет', 'пок'], ['привет', 'пок'])

In [158]:
#nlp = simple_NLP()
#doc_to_parsing = {}
#for line_info in tqdm(json_data.items()):
#    doc_to_parsing[int(line_info[0])] = nlp(line_info[1])

#print(len(doc_to_parsing))

In [159]:
doc_to_title = {}
with open('docs_titles.tsv',encoding='utf-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))

28026


In [160]:
train_data

,pair_id,group_id,doc_id,target
0,1,1,15731,0
1,2,1,14829,0
2,3,1,15764,0
3,4,1,17669,0
4,5,1,14852,0
...,...,...,...,...
11685,11686,129,26672,0
11686,11687,129,25838,0
11687,11688,129,25703,0
11688,11689,129,27885,0


In [161]:
#nlp(doc_to_title[123]) + texts[27037]

In [162]:
len(doc_to_title)

28026

In [163]:
len(train_data)

11690

In [167]:
len(urls)

28026

In [164]:
#import fasttext

#from sklearn.metrics.pairwise import cosine_similarity

In [169]:
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    #title = doc_to_parsing[doc_id].copy()
    #title.extend(nlp(doc_to_title[doc_id]))
    title = nlp(doc_to_title[doc_id])
    url = urls[doc_id - 1]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, url, target))

In [170]:
traingroups_titledata[129]

[(27384, ['lovecr'], ['rdfo', 'ru', 'menu'], 0),
 (27477,
  ['блог', 'серге', 'корол'],
  ['aeroshuttletransf',
   'org',
   'bilet',
   'iz',
   'ukraini',
   'v',
   'kanadu',
   'tsena',
   'xml'],
  0),
 (26126,
  ['сми', 'ит', 'рф', 'рубеж'],
  ['orthodox', 'su', 'index', 'php', 'topic', 'bd', 'ba', 'page'],
  0),
 (27084,
  ['росс', 'помощ', 'малообеспечен', 'онкобольн'],
  ['youtub', 'com', 'watch', 'v'],
  0),
 (27716,
  ['милосерд', 'ру', 'троицк', 'благочин', 'г', 'москв'],
  ['liveinternet', 'ru', 'user'],
  0),
 (27204,
  ['заработок', 'интернет', 'главн', 'страниц'],
  ['otvet', 'mail', 'ru', 'question'],
  0),
 (27400,
  ['форум',
   'социальн',
   'сет',
   'секрет',
   'инструкц',
   'рекомендац',
   'instagram'],
  ['volgograd', 'news', 'citi', 'info', 'doc', 'sistemsc', 'htm'],
  0),
 (26886,
  ['городск',
   'информацион',
   'методическ',
   'центр',
   'город',
   'владимир',
   'newtonew',
   'com'],
  ['studfil', 'ru', 'preview', 'page'],
  0),
 (27588,
  ['мам',

In [171]:
y_train = []
y_train1 = []
X_train = []
groups_train = []

for new_group in tqdm(traingroups_titledata):
    docs = traingroups_titledata[new_group]
    #print(docs)
    
    for k, (doc_id, title, text, target_id) in enumerate(docs):
        # exit()
        #y_train.append("__label__" + str(target_id))
        y_train.append(target_id)
        groups_train.append(new_group)
        #X_train.append(" ".join(title))
        all_dist1 = []
        all_dist2 = []
        words1 = set(title)
        words2 = set(text)
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, text_j, target_j = docs[j]
            words1_j = set(title_j)
            words2_j = set(text_j)
            all_dist1.append(len(words1.intersection(words1_j)))
            all_dist2.append(len(words2.intersection(words2_j)))
        #X_train.append(sorted(all_dist, reverse=True)[0:15] )
        first_features = sorted(all_dist1, reverse=True)[0:15]
        second_features = sorted(all_dist2, reverse=True)[0:15]
        X_train.append(first_features + second_features)
        #X_train.append(first_features)
        
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
#print(X_train.shape, y_train.shape, groups_train.shape)

In [172]:
#train_dataset = pd.DataFrame()
#train_dataset["text"] = X_train
#train_dataset["label"] = y_train
#train_dataset

In [173]:
#train_dataset.to_csv("train_fasttext.txt", sep='\t', header=None, index=None)

In [174]:
#%%time
#model_big = fasttext.train_supervised(
#    input="train_fasttext.txt",
#    minn=3, maxn=5,  # диапазон для символьных нграмм
#    wordNgrams=2,  # используем словесные нграммы размера 2
#    dim=20  # размер вектора
#) # логи обучения пишутся в терминале

In [175]:
#model_big.get_word_vector("пок")

In [137]:
#title1 = ['какой', 'сложный', 'проект']
#title2 = ['какой', 'легкий', 'проект']
#embed1 = model_big.get_sentence_vector(" ".join(title1))
#embed2 = model_big.get_sentence_vector(" ".join(title2))

In [176]:
#embed1

In [139]:
#cosine_similarity(embed1.reshape(1,-1), embed2.reshape(1,-1))[0][0]

0.99608797

In [177]:
#X_train = []
#groups_train = []
#
#for new_group in tqdm(traingroups_titledata):
#    docs = traingroups_titledata[new_group]
#    #print(docs)
#    
#    for k, (doc_id, title, target_id) in enumerate(docs):
#        groups_train.append(new_group)
#        #X_train.append(title)
#        all_dist1 = []
#        #all_dist2 = []
#        embed = model_big.get_sentence_vector(" ".join(title))
#        #words2 = set(text)
#        for j in range(0, len(docs)):
#            if k == j:
#                continue
#            doc_id_j, title_j, target_j = docs[j]
#            embed_j = model_big.get_sentence_vector(" ".join(title_j))
#        #    #words2_j = set(text_j)
#            #all_dist1.append(len(words1.intersection(words1_j)))
#            all_dist1.append(cosine_similarity(embed.reshape(1,-1), embed_j.reshape(1,-1))[0][0])
#        #    #all_dist2.append(len(words2.intersection(words2_j)))
#        X_train.append(sorted(all_dist1, reverse=True)[0:15] )
#        #first_features = sorted(all_dist1, reverse=True)[0:15]
#        #second_features = sorted(all_dist2, reverse=True)[0:15]
#        #X_train.append(first_features + second_features)
#        #X_train.append(first_features)
#        
#X_train = np.array(X_train)
#y_train = np.array(y_train1)
#groups_train = np.array(groups_train)
##print(X_train.shape, y_train.shape, groups_train.shape)

In [178]:
#true_mask = np.where(y_train == 1)
#not_true_mask = np.where(y_train != 1)
#true_mask

In [179]:
#np.mean(X_train[true_mask]) , np.mean(X_train[not_true_mask])

In [180]:
X_train[0], y_train[0]

(array([5, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2]),
 0)

In [181]:
X_train[28], y_train[28]

(array([4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 4, 4, 3, 3, 3, 3,
        2, 2, 2, 2, 2, 2, 2, 2]),
 1)

In [182]:
#from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer 

#vectorizer = TfidfVectorizer(min_df = 5, max_df = 0.8)

In [183]:
#X_train = list(map(lambda x: " ".join(x), X_train))

In [184]:
#X_train_vect = vectorizer.fit_transform(X_train)

In [185]:
X_train[0]

array([5, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

In [186]:
param_grid = {'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.03, 0.02, 0.1, 0.5],
              'n_estimators' : [50, 70, 100, 150, 200],
              'max_depth': [3,4,5,6,8,10]}

#param_grid = {'learning_rate': [0.1, 0.5],
#              'n_estimators' : [50],
#              'max_depth': [8]}
                
                
clf = xgb.XGBClassifier()
grid_search = GridSearchCV(clf, param_grid, cv=GroupKFold(n_splits=3), scoring = make_scorer(f1_score))

In [187]:
grid_search.fit(X_train, y_train, groups_train)  # Считает почти 10 минут. Сначала стоит попробовать {'learning_rate': 0.5, 'max_depth': 8, 'n_estimators': 200}

c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[  1   1   1 ... 129 129 129] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:12:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:12:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:12:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:12:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[17:12:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:12:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:12:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:12:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[17:13:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:13:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:13:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:13:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[17:13:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:13:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:13:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:13:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

...

[17:18:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[17:18:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[17:18:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[17:18:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

GridSearchCV(cv=GroupKFold(n_splits=3),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_...ors=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     

In [188]:
grid_search.best_params_

{'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 50}

In [189]:
best_clf =  grid_search.best_estimator_

In [190]:
cross_val_score(best_clf, X_train, y_train, groups=groups_train, scoring=make_scorer(f1_score)).mean()

c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:18:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:18:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

0.6605704024626411

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}



#param_grid = {'learning_rate': [0.1, 0.5],
#              'n_estimators' : [50],
#              'max_depth': [8]}
                
                
rf_clf = RandomForestClassifier()
grid_search = GridSearchCV(rf_clf, param_grid, cv=GroupKFold(n_splits=3), scoring = make_scorer(f1_score))

In [36]:
grid_search.fit(X_train, y_train, groups_train)

c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[  1   1   1 ... 129 129 129] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


GridSearchCV(cv=GroupKFold(n_splits=3), estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [False], 'criterion': ['gini'],
                         'max_depth': [None], 'max_features': [1, 3, 10],
                         'min_samples_leaf': [1, 3, 10],
                         'min_samples_split': [2, 3, 10],
                         'n_estimators': [100, 300]},
             scoring=make_scorer(f1_score))

In [37]:
best_rf_clf =  grid_search.best_estimator_

In [38]:
cross_val_score(best_rf_clf, X_train, y_train, groups=groups_train, scoring=make_scorer(f1_score)).mean()

0.6757681762268104

In [44]:
from sklearn.svm import SVC

In [47]:
param_grid = {"C": [1, 5, 10, 20, 30, 40, 50, 60 , 70, 80, 90, 100]}



#param_grid = {'learning_rate': [0.1, 0.5],
#              'n_estimators' : [50],
#              'max_depth': [8]}
                
                
svm_clf = SVC()
grid_search = GridSearchCV(svm_clf, param_grid, cv=GroupKFold(n_splits=3), scoring = make_scorer(f1_score))

In [48]:
grid_search.fit(X_train, y_train, groups_train)

c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[  1   1   1 ... 129 129 129] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


GridSearchCV(cv=GroupKFold(n_splits=3), estimator=SVC(),
             param_grid={'C': [1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]},
             scoring=make_scorer(f1_score))

In [49]:
best_svm_clf =  grid_search.best_estimator_

In [50]:
cross_val_score(best_svm_clf, X_train, y_train, groups=groups_train, scoring=make_scorer(f1_score)).mean()

0.6747506376925437

In [52]:

from sklearn.ensemble import StackingClassifier

In [65]:
estimators = [('rf', best_rf_clf), ('xgb', best_clf), ('svm', best_svm_clf)]

In [66]:
param_grid = {}



#param_grid = {'learning_rate': [0.1, 0.5],
#              'n_estimators' : [50],
#              'max_depth': [8]}
                
                
stack = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier())
grid_search = GridSearchCV(stack, param_grid, cv=GroupKFold(n_splits=3), scoring = make_scorer(f1_score))

In [67]:
grid_search.fit(X_train, y_train, groups_train)

c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass groups=[  1   1   1 ... 129 129 129] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:25:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:25:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:25:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:25:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:25:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:25:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:25:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:25:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:25:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:25:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

GridSearchCV(cv=GroupKFold(n_splits=3),
             estimator=StackingClassifier(estimators=[('rf',
                                                       RandomForestClassifier(bootstrap=False,
                                                                              max_features=1,
                                                                              min_samples_leaf=10)),
                                                      ('xgb',
                                                       XGBClassifier(base_score=0.5,
                                                                     booster='gbtree',
                                                                     colsample_bylevel=1,
                                                                     colsample_bynode=1,
                                                                     colsample_bytree=1,
                                                                     gamma=0,
                             

In [68]:
best_stack =  grid_search.best_estimator_

In [69]:
cross_val_score(best_stack, X_train, y_train, groups=groups_train, scoring=make_scorer(f1_score)).mean()

c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:25:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:25:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:26:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:26:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:26:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:26:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\денис\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:26:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:26:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:26:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:26:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

0.6325405136503439

In [54]:
#q = X_train_vect[0].toarray()
#q.mean()

## Отправляем

In [60]:
test_data = pd.read_csv('test_groups.csv')
testgroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = nlp(doc_to_title[doc_id])
    try:
        text = texts[doc_id - 1]
    except:
        print(doc_id)
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title, text))

In [67]:
X_test = []
groups_test = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title, text) in enumerate(docs):
        groups_test.append(new_group)
        all_dist1 = []
        all_dist2 = []
        words1 = set(title)
        words2 = set(text)
    
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, text_j = docs[j]
            words1_j = set(title_j)
            words2_j = set(text_j)
            all_dist1.append(len(words1.intersection(words1_j)))
            all_dist2.append(len(words2.intersection(words2_j)))
            #X_train.append(sorted(all_dist, reverse=True)[0:15] )
        first_features = sorted(all_dist1, reverse=True)[0:15]
        second_features = sorted(all_dist2, reverse=True)[0:15]
        X_test.append(first_features + second_features)
X_test = np.array(X_test)
groups_test = np.array(groups_test)
print(X_test.shape, groups_test.shape)


(16627, 30) (16627,)


In [68]:
X_test[0]

array([  3,   3,   3,   3,   2,   2,   2,   2,   2,   2,   2,   2,   1,
         1,   1, 205, 201, 180, 167, 165, 164, 159, 159, 152, 152, 150,
       147, 145, 141, 137])

In [69]:
y_pred = best_clf.predict(X_test)
y_pred = y_pred.astype(int)

In [70]:
y_pred.mean()

0.2884465026763698

In [71]:
sub = pd.DataFrame({'pair_id': np.asarray(test_data['pair_id']), 'target': y_pred})
sub = sub.set_index(['pair_id'])
sub.to_csv('submission.csv')

## Score = 0.66101